In [189]:
from aocd.models import Puzzle


def parses(input):
    input = [list(line) for line in input.strip().split()]
    return input
    

puzzle = Puzzle(year=2022, day=24)
data = parses(puzzle.input_data)

In [190]:
sample = parses("""#.######
#>>.<^<#
#.<..<<#
#>v.><>#
#<^v^^>#
######.#""")

In [152]:
def add(a, b):
    return (a[0]+b[0], a[1]+b[1])

def manhattan(a, b):
    return abs(a[0]-b[0]) + abs(a[1]-b[1])

In [153]:
from heapq import heappop, heappush

In [154]:
def precompute_blizzard(data):
    directions = np.array([(0,1),(0,-1),(1,0),(-1,0)])
    inside = np.array(data)[1:-1,1:-1]
    H, W = len(inside), len(inside[0])
    winds_init = []
    winds_dir = []
    for i, c in enumerate('><v^'):
        loc = np.argwhere(inside == c)
        winds_init.append(loc)
        winds_dir.append(np.broadcast_to(directions[i],(len(loc),2)))
    winds_init = np.vstack(winds_init)
    winds_dir = np.vstack(winds_dir)

    T = np.lcm(H,W)
    clear = []
    for t in range(T):
        winds_t = (winds_init + t * winds_dir) % np.array((H, W))
        clear.append(
            set([(x,y) for x in range(H) for y in range(W)]) -
            set([(x,y) for x,y in winds_t])
        )
    return clear, (H, W)

In [159]:
# def solve_a(data):
#     directions = [(0,1),(0,-1),(1,0),(-1,0)]
#     clear, (H, W) = precompute_blizzard(data)
    
#     start = -1, data[0].index('.') - 1
#     end = H, data[-1].index('.') - 1
    
#     visited = set([(start,)])
    
#     heap = [(manhattan(start, end), (start,))]
    
#     while heap:
# #         print(heap[-1])
#         cost, path = heappop(heap)
#         t = len(path) % len(clear)
#         *_, pos = path
        
#         for d in directions:
#             next_pos = add(pos, d)
#             if next_pos == end:
# #                 return len(path) + 1
#                 return path
#             if next_pos in clear[t]:
#                 new_path = path+(next_pos,)
#                 new_cost = len(new_path)+manhattan(next_pos, end)
#                 new_state = (new_cost, new_path )
#                 if new_state not in visited:
#                     visited.add(new_state)
#                     heappush(heap, new_state)
            
#         # wait
#         new_path = path+(pos,)
#         heappush(heap, (cost+1, new_path))

In [160]:
def solve_a(data):
    directions = [(0,1),(0,-1),(1,0),(-1,0), (0,0)]
    clear, (H, W) = precompute_blizzard(data)
    
    start = -1, data[0].index('.') - 1
    end = H, data[-1].index('.') - 1
    
    visited = set()
    
    heap = [(manhattan(start, end), start, 0)]
    T = len(clear)
    
    while heap:
        cost, pos, t = heappop(heap)
        
        for d in directions:
            next_pos = add(pos, d)
            if next_pos == end:
                return t+1
            if next_pos in clear[(t+1) % T]:
                new_cost = t+1+manhattan(next_pos, end)
                new_state = (new_cost, next_pos, t+1)
                if new_state not in visited:
                    visited.add(new_state)
                    heappush(heap, new_state)

In [161]:
solve_a(sample)

18

In [162]:
solve_a(data)

260

In [167]:
def solve_b(data):

    directions = [(0,1),(0,-1),(1,0),(-1,0), (0,0)]
    clear, (H, W) = precompute_blizzard(data)
    

    
    start = -1, data[0].index('.') - 1
    end = H, data[-1].index('.') - 1
    
    for clear_t in clear:
        clear_t |= set([start, end])
        
    T = len(clear)
    
    def fastest(start, end, offset):
        visited = set()

        heap = [(manhattan(start, end), start, 0)]

        while heap:
            cost, pos, t = heappop(heap)

            for d in directions:
                next_pos = add(pos, d)
                if next_pos == end:
                    return t+1
                if next_pos in clear[(t+1+offset) % T]:
                    new_cost = t+1+manhattan(next_pos, end)
                    new_state = (new_cost, next_pos, t+1)
                    if new_state not in visited:
                        visited.add(new_state)
                        heappush(heap, new_state)
    
    t1 = fastest(start, end, 0)
    t2 = fastest(end, start, t1)
    t3 = fastest(start, end, t1+t2)
    
    return t1+t2+t3

In [168]:
solve_b(sample)

54

In [169]:
%%time
solve_b(data)

CPU times: user 4.09 s, sys: 98.3 ms, total: 4.19 s
Wall time: 4.21 s


747

In [170]:
%%time
clear, (H, W) = precompute_blizzard(data)

CPU times: user 3.11 s, sys: 81.2 ms, total: 3.19 s
Wall time: 3.21 s


In [50]:
# np.lcm(H,W)

In [192]:
# start, end, clear_rows, clear_cols, H, W = precompute_blizzard(sample)

In [193]:
# sorted(clear_rows.items())

In [194]:
# sorted(clear_cols.items())

In [195]:
# viz = ''
# offset, t = 0, 9
# for r in range(-1,H+1):
#     for c in range(-1,W+1):

        
#         if empty_col and empty_row:
#             viz += '.'
#         else:
#             viz += '#'
#     viz += '\n'
# print(viz)

In [201]:
def precompute_blizzard(data):

    
    return start, end, dict(clear_rows), dict(clear_cols), H, W

In [220]:
def solve(data, part):
    # precompute positions without blizzard at each timestep t
    # We decompose vertical and horizontal blizzards separately
    # so we don't have to deal with LCM(H,W) period but with
    # H and W separately
    directions = np.array([(1,0),(-1,0),(0,1),(0,-1)])
    inside = np.array(data)[1:-1,1:-1]
    H, W = len(inside), len(inside[0])

    clear_cols = defaultdict(lambda: set(range(W))) # index r, t
    clear_rows = defaultdict(lambda: set(range(H))) # index c, t
    
    start = -1, data[0].index('.') - 1
    end = H, data[-1].index('.') - 1
    for i, c in enumerate('v^><'):
        winds_init = np.argwhere(inside == c)

        if i < 2:
            for t in range(H):
                for r, c in winds_init + t * directions[i]:      
                    clear_rows[c,t].discard(r%H)
        else:
            for t in range(W):
                for r, c in winds_init + t * directions[i]:      
                    clear_cols[r,t].discard(c%W)

    for r, c in (start, end):
        for t in range(W):     
            clear_cols[r, t] = set([c])
        for t in range(H):
            clear_rows[c, t] |= set([r])
    clear_rows, clear_cols = dict(clear_rows), dict(clear_cols)
    
    
    ## Perform A* three times
    directions = [(0,1),(0,-1),(1,0),(-1,0),(0,0)] # add no-op

    def fastest(start, end, offset):
        visited = set((start, 0))

        heap = [(manhattan(start, end), start, 0)]

        while heap:
            cost, pos, t = heappop(heap)

            for d in directions:
                next_pos = add(pos, d)
                if next_pos == end:
                    return t+1
                r, c = next_pos
                tr = (t+1+offset)%H
                tc = (t+1+offset)%W
                empty_row = (c,tr) in clear_rows and r in clear_rows[c,tr] 
                empty_col = (r,tc) in clear_cols and c in clear_cols[r,tc]
                if empty_row and empty_col:
                    new_cost = t+1+manhattan(next_pos, end)
                    new_state = (new_cost, next_pos, t+1)
                    if (next_pos, t+1) not in visited:
                        visited.add((next_pos, t+1))
                        heappush(heap, new_state)
    
    t1 = fastest(start, end, 0)
    if part == 'a':
        return t1
    elif part == 'b':
        t2 = fastest(end, start, t1)
        t3 = fastest(start, end, t1+t2)

        return t1 + t2 + t3

In [230]:
solve(sample, 'a')

18

In [231]:
solve(sample, 'b')

54

In [232]:
solve(data, 'a')

260

In [233]:
%%time
solve(data, 'b')

CPU times: user 3.73 s, sys: 31.1 ms, total: 3.76 s
Wall time: 3.79 s


747

In [197]:
solve_b(sample)

(18, 23, 13, 54)

In [187]:
%%time
solve_b(data)

CPU times: user 2.2 s, sys: 28.3 ms, total: 2.23 s
Wall time: 2.24 s


(260, 239, 248, 747)

In [188]:
%%time
_ = precompute_blizzard(data)

CPU times: user 310 ms, sys: 6.56 ms, total: 316 ms
Wall time: 314 ms


In [227]:
from collections import deque

In [229]:
def solve(data, part):
    # precompute positions without blizzard at each timestep t
    # We decompose vertical and horizontal blizzards separately
    # so we don't have to deal with LCM(H,W) period but with
    # H and W separately
    directions = np.array([(1,0),(-1,0),(0,1),(0,-1)])
    inside = np.array(data)[1:-1,1:-1]
    H, W = len(inside), len(inside[0])

    clear_cols = defaultdict(lambda: set(range(W))) # index r, t
    clear_rows = defaultdict(lambda: set(range(H))) # index c, t
    
    start = -1, data[0].index('.') - 1
    end = H, data[-1].index('.') - 1
    for i, c in enumerate('v^><'):
        winds_init = np.argwhere(inside == c)

        if i < 2:
            for t in range(H):
                for r, c in winds_init + t * directions[i]:      
                    clear_rows[c,t].discard(r%H)
        else:
            for t in range(W):
                for r, c in winds_init + t * directions[i]:      
                    clear_cols[r,t].discard(c%W)

    for r, c in (start, end):
        for t in range(W):     
            clear_cols[r, t] = set([c])
        for t in range(H):
            clear_rows[c, t] |= set([r])
    clear_rows, clear_cols = dict(clear_rows), dict(clear_cols)
    
    
    ## Perform A* three times
    directions = [(0,1),(0,-1),(1,0),(-1,0),(0,0)] # add no-op

    def fastest(start, end, offset):
        visited = set((start, 0))

        queue = deque([(manhattan(start, end), start, 0)])

        while queue:
            cost, pos, t = queue.popleft()

            for d in directions:
                next_pos = add(pos, d)
                if next_pos == end:
                    return t+1
                r, c = next_pos
                tr = (t+1+offset)%H
                tc = (t+1+offset)%W
                empty_row = (c,tr) in clear_rows and r in clear_rows[c,tr] 
                empty_col = (r,tc) in clear_cols and c in clear_cols[r,tc]
                if empty_row and empty_col:
                    new_cost = t+1+manhattan(next_pos, end)
                    new_state = (new_cost, next_pos, t+1)
                    if (next_pos, t+1) not in visited:
                        visited.add((next_pos, t+1))
                        queue.append(new_state)
    
    t1 = fastest(start, end, 0)
    if part == 'a':
        return t1
    elif part == 'b':
        t2 = fastest(end, start, t1)
        t3 = fastest(start, end, t1+t2)

        return t1 + t2 + t3